1) raw dataset
2) keywords based extraction
3) train/test split
4) sample a specific number of samples (high cardinality)
5) paste the instruction

In [1]:
from huggingface_hub import logout, notebook_login
# logout()
notebook_login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
## dataset load

from datasets import load_dataset

# qa datasets
medqa = load_dataset("GBaker/MedQA-USMLE-4-options")
# medqa = load_dataset("bigbio/med_qa")
medmcqa = load_dataset("openlifescienceai/medmcqa")
pubmedqa = load_dataset("bigbio/pubmed_qa", trust_remote_code=True)
# pubmedqa = load_dataset("qiaojin/PubMedQA", "pqa_artificial")
# bioasq = load_dataset("rag-datasets/rag-mini-bioasq", "question-answer-passages")
# mednli = load_dataset("bigbio/mednli")

# nli datasets
# mednli = load_dataset("cnut1648/mnli_resampled_as_mednli")
bionli = load_dataset("clinicalnlplab/BioNLI_test")

# information extraction datasets
# cie_mse = load_dataset("mitclinicalml/clinical-ie", "medication_status", trust_remote_code=True) # medication status extraction
# cie_ccr = load_dataset("mitclinicalml/clinical-ie", "coreference", trust_remote_code=True) # clinical coreference resolution

# medmentions = load_dataset("bigbio/medmentions", trust_remote_code=True)
# bc5cdr = load_dataset("bigbio/bc5cdr", trust_remote_code=True)

# chemdisgene = load_dataset("bigbio/chem_dis_gene", trust_remote_code=True)
# biored = load_dataset("bigbio/biored", trust_remote_code=True)
# biorelex = load_dataset("bigbio/biorelex", trust_remote_code=True)
biorel = load_dataset("DFKI-SLT/BioRel")
# biorex = load_dataset("bigbio/biorelex", trust_remote_code=True)

# generation task about clinical skills datasets
# medichat = load_dataset("Mostafijur/medichat_conversation", "medichat_subset1") # ~ medichat_subset15 # conv2note
icliniq = load_dataset("lavita/ChatDoctor-iCliniq")
# mediqa = load_dataset("jonathankang/EN-MEDIQA")
pubmed = load_dataset("ccdv/pubmed-summarization")

In [3]:
icliniq = icliniq['train'].train_test_split(test_size=0.3, seed=42)
# pubmedqa = pubmedqa['train'].train_test_split(test_size=0.3, seed=42)

In [4]:
print("medqa")
print(medqa)
print('------------------------------------------------------------------------------'* 2) 
print("medmcqa")
print(medmcqa)
print('------------------------------------------------------------------------------'* 2) 
print("pubmedqa")
print(pubmedqa)
print('------------------------------------------------------------------------------'* 2) 
# print("mednli")
# print(mednli)
# print('------------------------------------------------------------------------------'* 2) 
print("bionli")
print(bionli)
print('------------------------------------------------------------------------------'* 2) 
# print("cie_mse")
# print(cie_mse)
# print('------------------------------------------------------------------------------'* 2) 
# print("cie_ccr")
# print(cie_ccr)
# print('------------------------------------------------------------------------------'* 2) 
# print("medmentions")
# print(medmentions)
# print('------------------------------------------------------------------------------'* 2) 
# print("bc5cdr")
# print(bc5cdr)
# print('------------------------------------------------------------------------------'* 2) 
# print("chem-dis-gene")
# print(chemdisgene)
# print('------------------------------------------------------------------------------'* 2) 
# print("biored")
# print(biored)
# print('------------------------------------------------------------------------------'* 2) 
# print("biorelex")
# print(biorelex)
# print('------------------------------------------------------------------------------'* 2) 
print("biorel")
print(biorel)
print('------------------------------------------------------------------------------'* 2) 
# print("biorex")
# print(biorex)
# print('------------------------------------------------------------------------------'* 2) 
# print("medichat")
# print(medichat)
# print('------------------------------------------------------------------------------'* 2) 
print("icliniq")
print(icliniq)
print('------------------------------------------------------------------------------'* 2) 
# print("mediqa")
# print(mediqa)
# print('------------------------------------------------------------------------------'* 2) 
print("pubmed")
print(pubmed)

medqa
DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 1273
    })
})
------------------------------------------------------------------------------------------------------------------------------------------------------------
medmcqa
DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_na

In [5]:
import pandas as pd
import os

 
keywords = [
    # 질환 관련
    "diabetes", "HbA1c", "blood sugar", 
    "glucose", "ketoacidosis",
    
    # 약물 및 치료
    "insuline",
    "metformin", "SGLT2 inhibitors", "GLP-1 receptor agonist",

    # 합병증
    "neuropathy", "retinopathy", "nephropathy",
]

def contains_keywords(text, keywords):
    if not text or not isinstance(text, str): 
        return False
    return any(keyword in text.lower() for keyword in keywords)

def process_dataset_split(dataset_name, split_name, dataset, text_columns, output_file, keywords=["diabetes", "insulin"]):
    rows = []
    for i in range(len(dataset)):
        row = dataset[i]

        for col in text_columns:
            if col in row:
                text = row[col]

            elif col == "abstract" and "passages" in row:
                if isinstance(row["passages"], list):
                    text = " ".join([p["text"] for p in row["passages"] if p.get("type") == "abstract"])
                else:
                    text = ""
            else:
                text = ""

            if not text:
                continue

            is_related = 1 if contains_keywords(text, keywords) else 0

            rows.append({
                "dataset": dataset_name,
                "split_data": split_name,
                "features": row,  
                "input": text,
                "output": is_related
            })

    if rows:  
        df = pd.DataFrame(rows)
        df.to_csv(output_file, mode="a", header=not os.path.exists(output_file), index=False)
        print(f"{dataset_name} ({split_name}): {len(rows)} rows processed and appended to {output_file}")
    else:
        print(f"{dataset_name} ({split_name}): No rows processed.")

datasets = {
    "medqa": {"splits": medqa, "columns": ["question"]},
    "medmcqa": {"splits": medmcqa, "columns": ["question"]},
    "pubmedqa": {"splits": pubmedqa, "columns": ["QUESTION"]},
    # "mednli": {"splits": mednli, "columns": ["premise", "hypothesis"]},
    "bionli": {"splits": bionli, "columns": ["query", "answer"]},
    # "cie_mse": {"splits": cie_mse, "columns": ["snippet"]},
    # "cie_ccr": {"splits": cie_ccr, "columns": ["snippet"]},
    # "medmentions: {"splits": medmentions, "columns": ["passages"]},
    # "bc5cdr": {"splits": bc5cdr, "columns": ["passages"]},
    # "chemdisgene": {"splits": chemdisgene, "columns": ["passages"]},
    # "biored": {"splits": biored, "columns": ["passages"]},
    # "biorelex": {"splits": biorelex, "columns": ["text"]},
    "biorel": {"splits": biorel, "columns": ["text"]},
    # "biorex": {"splits": biorex, "columns": ["text"]},
    # "medichat": {"splits": medichat, "columns": ["conversation"]},
    "icliniq": {"splits": icliniq, "columns": ["input", "answer_icliniq"]},
    "pubmed": {"splits": pubmed, "columns": ["abstract"]},
}

output_file = os.path.join("all_data_filtered_by_keywords.csv")

for dataset_name, details in datasets.items():
    splits = details["splits"]
    text_columns = details["columns"] 

    for split_name, split_data in splits.items():
        process_dataset_split(dataset_name, split_name, split_data, text_columns, output_file, keywords=keywords)


medqa (train): 10178 rows processed and appended to all_data_filtered_by_keywords.csv
medqa (test): 1273 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (train): 182822 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (test): 6150 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (validation): 4183 rows processed and appended to all_data_filtered_by_keywords.csv
pubmedqa (train): 200000 rows processed and appended to all_data_filtered_by_keywords.csv
pubmedqa (validation): 11269 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (train): 11088 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (validation): 25612 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (test): 12616 rows processed and appended to all_data_filtered_by_keywords.csv
biorel (train): 534277 rows processed and appended to all_data_filtered_by_keywords.csv
biorel (validation): 11450

In [1]:
import pandas as pd

all_df = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/all_data_filtered_by_keywords.csv")

In [2]:
# data_filtered_by_keyword
diabetes = all_df[all_df['output'] == 1]

In [3]:
diabetes.head(2)

,dataset,split_data,features,input,output
15,medqa,train,{'question': 'A 68-year-old man presents to th...,A 68-year-old man presents to the emergency de...,1
17,medqa,train,{'question': 'A 68-year-old man comes to the p...,A 68-year-old man comes to the physician becau...,1


In [4]:
diabetes.groupby(['dataset', 'split_data'])['output'].agg('count')

dataset   split_data
bionli    test           3906
          train          3690
          validation     8280
biorel    test          15618
          train         69864
          validation    15396
biorelex  train             1
biorex    train             1
icliniq   test            972
          train          2238
medmcqa   test            414
          train         11952
          validation      270
medqa     test           1122
          train          9564
pubmed    test           3264
          train         54714
          validation     3222
pubmedqa  train         21744
          validation     1227
Name: output, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

def re_train_test_split_unique(dataframe, train_ratio=0.8):
    new_rows = []

    grouped = dataframe.groupby('dataset')

    for dataset_name, group in grouped:
        if dataset_name not in ['bionli', 'biorel', 'icliniq', 'medmcqa', 'medqa', 'pubmed', 'pubmedqa']:
            continue

        # 중복 제거
        group = group.drop_duplicates(subset='features')

        train_data, test_data = train_test_split(
            group,
            test_size=1-train_ratio,
            random_state=42,
        )

        train_data = train_data.copy()
        train_data['split_data'] = 'train'

        test_data = test_data.copy()
        test_data['split_data'] = 'test'

        new_rows.append(train_data)
        new_rows.append(test_data)

    result_df = pd.concat(new_rows, ignore_index=True)
    return result_df

diabetes2 = re_train_test_split_unique(diabetes, train_ratio=0.8)

In [6]:
diabetes2.groupby(['dataset', 'split_data'])['output'].agg('count')

dataset   split_data
bionli    test            426
          train          1701
biorel    test           3359
          train         13434
icliniq   test             89
          train           356
medmcqa   test            422
          train          1684
medqa     test            357
          train          1424
pubmed    test           2039
          train          8154
pubmedqa  test           1532
          train          6125
Name: output, dtype: int64

In [7]:
import ast
import pandas as pd

def add_input_to_for_sampling(dataframe):
    dataframe = dataframe.copy()
    
    # INPUT 값을 추출하여 새로운 컬럼에 추가
    def extract_for_sampling(row):
        if row['dataset'] == 'bionli':
            features = ast.literal_eval(row['features']) 
            query = features.get('query', '') 
            input_text = query.split("INPUT: ")[1].split("[HYP]")[0].strip() if "INPUT: " in query else None
            answer = features.get('answer', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'biorel':
            features = ast.literal_eval(row['features'])
            text = features.get('text', None)
            relation = features.get('relation', None)
            return {'input': text, 'output': relation}
        elif row['dataset'] == 'icliniq':
            features = ast.literal_eval(row['features'])
            input_text = features.get('input', None)
            answer_iclinq = features.get('answer_iclinq', None)
            return {'input': input_text, 'output': answer_iclinq}
        elif row['dataset'] == 'medmcqa':
            features = ast.literal_eval(row['features'])
            input_text = features.get('question', None)
            answer = ", ".join([
                features.get('opa', ''),
                features.get('opb', ''),
                features.get('opc', ''),
                features.get('opd', ''),
            ]).strip()
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'medqa':
            features = ast.literal_eval(row['features'])
            input_text = features.get('question', None)
            answer = features.get('answer', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'pubmed':
            features = ast.literal_eval(row['features'])
            input_text = features.get('article', None)
            abstract = features.get('abstract', None)
            return {'input': input_text, 'output': abstract}
        elif row['dataset'] == 'pubmedqa':
            features = ast.literal_eval(row['features'])
            question = features.get('QUESTION', None)
            answer = features.get('LONG_ANSWER', None)
            return {'input': question, 'output': answer}

    dataframe['for_sampling'] = dataframe.apply(extract_for_sampling, axis=1)
    return dataframe

diabetes3 = add_input_to_for_sampling(diabetes2)

In [8]:
diabetes3.head(2)

,dataset,split_data,features,input,output,for_sampling
0,bionli,train,"{'id': 'BioNLI16787330', 'query': '\nTASK: Ple...",\nTASK: Please classify the relationship betwe...,1,{'input': '[PRE] An increased oxidative stress...
1,bionli,train,"{'id': 'BioNLI22643846', 'query': '\nTASK: Ple...",\nTASK: Please classify the relationship betwe...,1,{'input': '[PRE] Previously we demonstrated th...


In [9]:
diabetes3[diabetes3['dataset']=='biorel'].head(2)

,dataset,split_data,features,input,output,for_sampling
2127,biorel,train,"{'text': 'hla antigens , complement allotypes ...","hla antigens , complement allotypes , insulin ...",1,"{'input': 'hla antigens , complement allotypes..."
2128,biorel,train,"{'text': ""glucosamine and n-acetylglucosamine ...",glucosamine and n-acetylglucosamine supported ...,1,{'input': 'glucosamine and n-acetylglucosamine...


In [10]:
len(diabetes3[diabetes3['dataset']=='biorel'])

16793

In [11]:
len(diabetes3)

41102

In [12]:
import pandas as pd
import ast 

def filter_na_relations(dataframe):
    def is_valid_relation(features_str):
        try:
            features_dict = ast.literal_eval(features_str)
            return features_dict.get('relation') is not None and features_dict['relation'] != 'NA'
        except Exception as e:
            return False

    biorel_filtered = dataframe[dataframe['dataset'] == 'biorel']
    biorel_filtered = biorel_filtered[biorel_filtered['features'].apply(is_valid_relation)]

    others = dataframe[dataframe['dataset'] != 'biorel']

    result = pd.concat([biorel_filtered, others], ignore_index=True)
    return result

diabetes3 = filter_na_relations(diabetes3)


In [13]:
len(diabetes3[diabetes3['dataset']=='biorel'])

12054

In [14]:
len(diabetes3)

36363

In [15]:
diabetes3[diabetes3['dataset']=='biorel']['features'].iloc[0]

"{'text': 'hla antigens , complement allotypes , insulin antibodies and thyrogastric autoantibodies were determined in 69 patients with type 1 ( insulin-dependent ) diabetes defined by a tendency to ketosis , non-obesity and insulin requirement within 2 years of diagnosis .', 'relation': 'may_be_treated_by', 'h': {'id': 'C0011854', 'name': 'insulin-dependent', 'pos': [134, 151]}, 't': {'id': 'C0021641', 'name': 'insulin', 'pos': [214, 221]}}"

In [16]:
len(diabetes3)

36363

In [17]:
import os
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
import random
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def mmr(query_embedding, doc_embeddings, diversity, top_n):
    faiss.normalize_L2(doc_embeddings)
    faiss.normalize_L2(query_embedding.reshape(1, -1))

    selected_indices = []
    candidate_indices = list(range(len(doc_embeddings)))

    for _ in range(top_n):
        if not candidate_indices:
            break

        if len(selected_indices) == 0:
            selected_idx = candidate_indices[np.argmax(np.dot(doc_embeddings[candidate_indices], query_embedding.T))]
        else:
            selected_embeddings = doc_embeddings[selected_indices]
            similarity_to_selected = np.dot(doc_embeddings[candidate_indices], selected_embeddings.T)
            diversity_scores = np.max(similarity_to_selected, axis=1)
            relevance_scores = np.dot(doc_embeddings[candidate_indices], query_embedding.T).flatten()
            mmr_scores = (1 - diversity) * relevance_scores - diversity * diversity_scores
            selected_idx = candidate_indices[np.argmax(mmr_scores)]

        selected_indices.append(selected_idx)
        candidate_indices.remove(selected_idx)

    return selected_indices

def mmr_sampling(dataframe, sampling_dict, embedding_model='all-MiniLM-L6-v2', batch_size=64, diversity=0.7, seed=42):
    set_seed(seed)

    print("Loading SentenceTransformer model on GPU...")
    model = SentenceTransformer(embedding_model, device="cuda:0")

    sampled_rows = []

    for dataset_name, sample_count in tqdm(sampling_dict.items(), desc="Processing datasets"):
        subset = dataframe[(dataframe['dataset'] == dataset_name) & (dataframe['split_data'] == 'train')].copy()
        num_rows = len(subset)
        if num_rows == 0:
            print(f"No data found for dataset: {dataset_name}")
            continue

        print(f"Generating embeddings for dataset: {dataset_name} ({num_rows} rows)")
        start_time = time.time()

        embeddings = model.encode(
            subset['for_sampling'].astype(str).tolist(),
            batch_size=batch_size,
            convert_to_numpy=True,
            show_progress_bar=True
        )

        elapsed_time = time.time() - start_time
        print(f"Embedding generation took {elapsed_time:.2f} seconds for {num_rows} rows")

        query_embedding = np.mean(embeddings, axis=0)

        print(f"Applying MMR for dataset: {dataset_name}")
        selected_indices = mmr(query_embedding, embeddings, diversity, sample_count)
        sampled_subset = subset.iloc[selected_indices]
        sampled_rows.append(sampled_subset)

    return pd.concat(sampled_rows, ignore_index=True)

# 샘플링 설정
sampling_config = {
    'bionli': 1000,
    'biorel': 1000,
    'icliniq': 300,
    'medmcqa': 1000,
    'medqa': 1000,
    'pubmedqa': 1000,
    'pubmed': 1000
}

set_seed(42)

# 샘플링 실행
sampled_train = mmr_sampling(diabetes3, sampling_config, batch_size=4096, diversity=0.7, seed=42)

# 결과 확인
print(sampled_train.head())
print(f"총 샘플링된 행 수: {len(sampled_train)}")

# 결과 저장
sampled_train.to_csv("final_combined_train_sample.csv")


/data/jaesung/anaconda3/envs/faiss_gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading SentenceTransformer model on GPU...


Processing datasets:   0%|          | 0/7 [00:00<?, ?it/s]

Generating embeddings for dataset: bionli (1701 rows)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


...

Generating embeddings for dataset: pubmed (8154 rows)


Batches: 100%|██████████| 2/2 [00:15<00:00,  7.71s/it]


Embedding generation took 23.65 seconds for 8154 rows
Applying MMR for dataset: pubmed


Processing datasets: 100%|██████████| 7/7 [01:28<00:00, 12.57s/it]


  dataset split_data                                           features  \
0  bionli      train  {'id': 'BioNLI18619553', 'query': '\nTASK: Ple...   
1  bionli      train  {'id': 'BioNLI30390733', 'query': "\nTASK: Ple...   
2  bionli      train  {'id': 'BioNLI21447232', 'query': '\nTASK: Ple...   
3  bionli      train  {'id': 'BioNLI21146692', 'query': "\nTASK: Ple...   
4  bionli      train  {'id': 'BioNLI25180937', 'query': '\nTASK: Ple...   

                                               input  output  \
0  \nTASK: Please classify the relationship betwe...       1   
1  \nTASK: Please classify the relationship betwe...       1   
2  \nTASK: Please classify the relationship betwe...       1   
3  \nTASK: Please classify the relationship betwe...       1   
4  \nTASK: Please classify the relationship betwe...       1   

                                        for_sampling  
0  {'input': '[PRE] Previously, we demonstrated t...  
1  {'input': '[PRE] Dear Editor, Eczema is an inf...  

In [18]:
import os
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
import random
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def mmr(query_embedding, doc_embeddings, diversity, top_n):
    faiss.normalize_L2(doc_embeddings)
    faiss.normalize_L2(query_embedding.reshape(1, -1))

    selected_indices = []
    candidate_indices = list(range(len(doc_embeddings)))

    for _ in range(top_n):
        if not candidate_indices:
            break

        if len(selected_indices) == 0:
            selected_idx = candidate_indices[np.argmax(np.dot(doc_embeddings[candidate_indices], query_embedding.T))]
        else:
            selected_embeddings = doc_embeddings[selected_indices]
            similarity_to_selected = np.dot(doc_embeddings[candidate_indices], selected_embeddings.T)
            diversity_scores = np.max(similarity_to_selected, axis=1)
            relevance_scores = np.dot(doc_embeddings[candidate_indices], query_embedding.T).flatten()
            mmr_scores = (1 - diversity) * relevance_scores - diversity * diversity_scores
            selected_idx = candidate_indices[np.argmax(mmr_scores)]

        selected_indices.append(selected_idx)
        candidate_indices.remove(selected_idx)

    return selected_indices

def mmr_sampling(dataframe, sampling_dict, embedding_model='all-MiniLM-L6-v2', batch_size=64, diversity=0.7, seed=42):
    set_seed(seed)

    print("Loading SentenceTransformer model on GPU...")
    model = SentenceTransformer(embedding_model, device="cuda:0")

    sampled_rows = []

    for dataset_name, sample_count in tqdm(sampling_dict.items(), desc="Processing datasets"):
        subset = dataframe[(dataframe['dataset'] == dataset_name) & (dataframe['split_data'] == 'test')].copy()
        num_rows = len(subset)
        if num_rows == 0:
            print(f"No data found for dataset: {dataset_name}")
            continue

        print(f"Generating embeddings for dataset: {dataset_name} ({num_rows} rows)")
        start_time = time.time()

        embeddings = model.encode(
            subset['for_sampling'].astype(str).tolist(),
            batch_size=batch_size,
            convert_to_numpy=True,
            show_progress_bar=True
        )

        elapsed_time = time.time() - start_time
        print(f"Embedding generation took {elapsed_time:.2f} seconds for {num_rows} rows")

        query_embedding = np.mean(embeddings, axis=0)

        print(f"Applying MMR for dataset: {dataset_name}")
        selected_indices = mmr(query_embedding, embeddings, diversity, sample_count)
        sampled_subset = subset.iloc[selected_indices]
        sampled_rows.append(sampled_subset)

    return pd.concat(sampled_rows, ignore_index=True)

# 샘플링 설정
sampling_config = {
    'bionli': 200,
    'biorel': 200,
    'icliniq': 60,
    'medmcqa': 200,
    'medqa': 200,
    'pubmedqa': 200,
    'pubmed': 200
}

set_seed(42)

# 샘플링 실행
sampled_test = mmr_sampling(diabetes3, sampling_config, batch_size=4096, diversity=0.7, seed=42)

# 결과 확인
print(sampled_test.head())
print(f"총 샘플링된 행 수: {len(sampled_test)}")

# 결과 저장
sampled_test.to_csv("final_combined_test_sample.csv")


Loading SentenceTransformer model on GPU...


Processing datasets:   0%|          | 0/7 [00:00<?, ?it/s]

Generating embeddings for dataset: bionli (426 rows)


Processing datasets:  14%|█▍        | 1/7 [00:00<00:03,  1.74it/s]

Embedding generation took 0.44 seconds for 426 rows
Applying MMR for dataset: bionli
Generating embeddings for dataset: biorel (2433 rows)


...

Generating embeddings for dataset: pubmed (2039 rows)


Batches: 100%|██████████| 1/1 [00:03<00:00,  3.60s/it]


Embedding generation took 5.66 seconds for 2039 rows
Applying MMR for dataset: pubmed


Processing datasets: 100%|██████████| 7/7 [00:10<00:00,  1.57s/it]


  dataset split_data                                           features  \
0  bionli       test  {'id': 'BioNLI9284904', 'query': '\nTASK: Plea...   
1  bionli       test  {'id': 'BioNLI22820188', 'query': '\nTASK: Ple...   
2  bionli       test  {'id': 'BioNLI29104632', 'query': '\nTASK: Ple...   
3  bionli       test  {'id': 'BioNLI7793231', 'query': "\nTASK: Plea...   
4  bionli       test  {'id': 'BioNLI12169771', 'query': '\nTASK: Ple...   

                                               input  output  \
0  \nTASK: Please classify the relationship betwe...       1   
1  \nTASK: Please classify the relationship betwe...       1   
2  \nTASK: Please classify the relationship betwe...       1   
3  \nTASK: Please classify the relationship betwe...       1   
4  \nTASK: Please classify the relationship betwe...       1   

                                        for_sampling  
0  {'input': '[PRE] Because of a failure to detec...  
1  {'input': '[PRE] Tumor necrosis factor-alpha (...  

In [19]:
import pandas as pd

final_combined_train_sample = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/final_combined_train_sample.csv")
final_combined_test_sample = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/final_combined_test_sample.csv")

In [20]:
# 중복 제거 후 겹치는 개수 확인
train_unique = final_combined_train_sample.drop_duplicates(subset='features')
test_unique = final_combined_test_sample.drop_duplicates(subset='features')

# train/test 겹치는 개수 계산
overlapping_count = len(pd.merge(train_unique, test_unique, on='features'))
print(f"Train/Test 겹치는 feature 개수: {overlapping_count}")


Train/Test 겹치는 feature 개수: 0


In [21]:
import numpy as np

final_combined_train_sample['task'] = None

# bionli
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'bionli', 'task'] = 'nli' 

# biorel
biorel_indices = final_combined_train_sample[final_combined_train_sample['dataset'] == 'biorel'].index
biorel_splits = np.array_split(biorel_indices, 1)

biorel_tasks = [
    'ie_extract_relation',
    # 'ie_classify_relation',
    # 'ie_identify_relation_exist',
    # 'ie_generate_relation_pair'
]

for split, task in zip(biorel_splits, biorel_tasks):
    final_combined_train_sample.loc[split, 'task'] = task

# icliniq
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'icliniq', 'task'] = 'generation'

# medmcqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'medmcqa', 'task'] = 'qa_subjective'

# medqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'medqa', 'task'] = 'qa_objective'

# pubmedqa
pubmedqa_indices = final_combined_train_sample[final_combined_train_sample['dataset'] == 'pubmedqa'].index
pubmedqa_splits = np.array_split(pubmedqa_indices, 1)

pubmedqa_tasks = [
    # 'qa_mesh_tagging',
    # 'qa_short_answer',
    'qa_descriptive',
    # 'qa_complex_answer'
]

for split, task in zip(pubmedqa_splits, pubmedqa_tasks):
    final_combined_train_sample.loc[split, 'task'] = task

# pubmed
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'pubmed', 'task'] = 'summarization'

In [22]:
import numpy as np

final_combined_test_sample['task'] = None

# bionli
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'bionli', 'task'] = 'nli' 

# biorel
biorel_indices = final_combined_test_sample[final_combined_test_sample['dataset'] == 'biorel'].index
biorel_splits = np.array_split(biorel_indices, 1)

biorel_tasks = [
    'ie_extract_relation',
    # 'ie_classify_relation',
    # 'ie_identify_relation_exist',
    # 'ie_generate_relation_pair'
]

for split, task in zip(biorel_splits, biorel_tasks):
    final_combined_test_sample.loc[split, 'task'] = task

# icliniq
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'icliniq', 'task'] = 'generation'

# medmcqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'medmcqa', 'task'] = 'qa_subjective'

# medqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'medqa', 'task'] = 'qa_objective'

# pubmedqa
pubmedqa_indices = final_combined_test_sample[final_combined_test_sample['dataset'] == 'pubmedqa'].index
pubmedqa_splits = np.array_split(pubmedqa_indices, 1)

pubmedqa_tasks = [
    # 'qa_mesh_tagging',
    # 'qa_short_answer',
    'qa_descriptive',
    # 'qa_complex_answer'
]

for split, task in zip(pubmedqa_splits, pubmedqa_tasks):
    final_combined_test_sample.loc[split, 'task'] = task

# pubmed
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'pubmed', 'task'] = 'summarization'


In [23]:
import random

instruction_mapping = {
    'nli': [
        "Please classify the relationship between the given premise and hypothesis into one of the following labels: entailment, contradiction, or neutral. return only the label."
        # "Given a premise and a hypothesis, determine their relationship: entailment, contradiction, or neutral.",
        # "Read the given premise and hypothesis. Decide if the hypothesis logically follows from the premise.",
        # "Classify the relationship between the premise and the hypothesis into one of three categories: entailment, contradiction, or neutral.",
        # "Analyze the relationship between the given premise and hypothesis. Categorize it as entailment, contradiction, or neutral.",
        # "Does the premise logically support the hypothesis? Answer as entailment, contradiction, or neutral.",
        # "Evaluate if the hypothesis can be inferred from the premise. Label it as entailment, contradiction, or neutral.",
        # "Does the hypothesis contradict the premise or is it entailed by it? If neither, classify it as neutral.",
        # "For the given premise and hypothesis, determine their logical relationship: entailment, contradiction, or neutral."
    ],
    'ie_extract_relation': [
        "Predict the relationship between the given entities in the given sentence. "
        # "Extract the relationship described between the given entities in the text.",
        # "Identify the type of relationship that connects the specified entities based on the provided text.",
        # "Determine the specific relationship mentioned between the entities in the text.",
        # "From the text, describe the relationship that exists between the provided entities."
    ],
    # 'ie_identify_relation_exist': [
    #     "Does the text describe any relationship between the specified entities? Answer Yes or No.",
    #     "Based on the text, determine if there is a connection or relationship between the given entities. Answer Yes or No.",
    #     "Identify whether a relationship exists between the specified entities in the text. Answer Yes or No.",
    #     "From the text, confirm whether there is a described relationship connecting the given entities. Answer Yes or No."
    # ],
    # 'ie_classify_relation': [
    #     "Classify the type of relationship described between the specified entities in the text.",
    #     "Identify the category that best describes the relationship between the entities based on the text.",
    #     "Determine whether the relationship described in the text falls into predefined categories such as physical, causal, or regulatory.",
    #     "Based on the text, categorize the relationship between the entities into its most relevant type."
    # ],
    # 'ie_generate_relation_pair': [
    #     "Generate all possible entity pairs from the text and identify their relationships.",
    #     "Extract all entity pairs mentioned in the text along with the relationships that connect them.",
    #     "Identify all entity pairs present in the text and specify their corresponding relationships if available.",
    #     "From the text, list all pairs of entities and describe the relationships that exist between them."
    # ],
    'generation': [
        # "What are the next steps or follow-up actions suggested in the doctor’s response?",
        "Generate a comprehensive medical response based on the patient's query.",
        # "Provide a concise and actionable answer to the patient's question.",
        # "What advice or recommendations does the doctor provide in response to the patient's concerns?",
        # "What follow-up steps does the doctor suggest in their response?"
    ],
    'qa_objective': [
        "Select the most appropriate answer for the given medical question from the provided options.",
        # "Identify the correct answer to the medical question from the four given options.",
        # "Pick the correct choice for the given medical question based on the provided options.",
        # "Determine the correct option that best answers the given medical question.",
        # "Choose the answer that accurately addresses the given medical query from the options provided.",
        # "From the provided choices, select the option that correctly answers the medical question.",
        # "Based on the medical question, select the most accurate answer from the given options.",
        # "Identify the best answer for the given medical question from the options below.",
        # "Select the correct answer that corresponds to the given medical question.",
        # "Determine the appropriate choice for the medical question from the list of options."
    ],
    'qa_subjective': [
        "Select the most appropriate answer for the given medical question from the provided options."
    ],
    # 'qa_mesh_tagging': [
    #     "Identify all MeSH terms mentioned in the context and list them.",
    #     "Extract relevant MeSH terms from the provided text that describe the main topics.",
    #     "Tag the context with appropriate MeSH terms that represent its primary themes.",
    #     "Using the context, identify the MeSH terms that best summarize its content.",
    #     "Highlight the MeSH terms in the text that are crucial for understanding the main subject.",
    #     "Using the given context, tag the text with relevant MeSH terms that describe its focus.",
    #     "From the context, extract key MeSH terms that align with the primary topics discussed.",
    #     "Analyze the text and identify all MeSH terms that pertain to the discussed topics."
    # ],
    # 'qa_complex_answer': [
    #     "Predict the correct answer (Yes, No, Maybe) for the given question and provide a detailed explanation based on the context.",
    #     "Determine whether the answer is Yes, No, or Maybe and explain your choice using evidence from the context.",
    #     "Using the provided context, predict the correct answer and write a detailed response to justify it.",
    #     "Select the correct decision (Yes, No, Maybe) for the given question and elaborate on the reasoning.",
    #     "Provide both the correct answer and a comprehensive explanation to the given question using the context.",
    #     "Answer the question with Yes, No, or Maybe, and support your decision with details from the context.",
    #     "Generate a Yes, No, or Maybe answer for the question and add a long-form response explaining why.",
    #     "Decide the answer to the medical question (Yes, No, Maybe) and provide a detailed justification using the given context.",
    #     "Using the context, predict the answer (Yes, No, Maybe) and generate a detailed explanation for the question.",
    #     "Write the correct decision (Yes, No, Maybe) for the question and support it with relevant findings from the context."
    # ],
    # 'qa_short_answer': [
    #     "Choose the correct answer (Yes, No, or Maybe) for the given question based on the provided context.",
    #     "Determine the correct decision (Yes, No, or Maybe) for the medical question using the given context.",
    #     "From the provided information, select whether the answer to the question is Yes, No, or Maybe.",
    #     "Analyze the context and identify whether the question should be answered with Yes, No, or Maybe.",
    #     "Based on the given data, decide if the answer to the question is Yes, No, or Maybe."
    # ],
    'qa_descriptive': [
        "Generate a detailed answer to the question using the provided context.",
        # "Create a comprehensive explanation for the question based on the given information.",
        # "Write a long and detailed response to the question using the context provided.",
        # "Using the context, provide an in-depth answer to the question with relevant details."
    ],
    'summarization': [
        # "Summarize the given article into a concise abstract that highlights the key findings and conclusions.",
        # "Generate a brief and coherent abstract from the provided article text.",
        "Write a summary of the article that captures the main ideas and significant details.",
        # "Condense the given article into a clear and concise abstract that represents its core content."
    ]
}

def assign_correct_instruction(row):
    task = row['task']
    if task in instruction_mapping:
        return random.choice(instruction_mapping[task])
    return "Instruction not found"

final_combined_train_sample['instruction'] = final_combined_train_sample.apply(assign_correct_instruction, axis=1)
final_combined_test_sample['instruction'] = final_combined_test_sample.apply(assign_correct_instruction, axis=1)

In [24]:
import ast
import pandas as pd

def add_input_to_for_sampling(dataframe):
    dataframe = dataframe.copy()
    
    # INPUT 값을 추출하여 새로운 컬럼에 추가
    def extract_for_sampling(row):
        if row['dataset'] == 'bionli':
            features = ast.literal_eval(row['features']) 
            query = features.get('query', '') 
            input_text = query.split("INPUT: ")[1].split("[HYP]")[0].strip() if "INPUT: " in query else None
            answer = features.get('answer', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'biorel':
            features = ast.literal_eval(row['features'])
            text = features.get('text', None)
            relation = features.get('relation', None)
            return {'input': text, 'output': relation}
        elif row['dataset'] == 'icliniq':
            features = ast.literal_eval(row['features'])
            input_text = features.get('input', None)
            answer_iclinq = features.get('answer_iclinq', None)
            return {'input': input_text, 'output': answer_iclinq}
        elif row['dataset'] == 'medmcqa':
            features = ast.literal_eval(row['features'])
            input_text = features.get('question', None)
            answer = ", ".join([
                features.get('opa', ''),
                features.get('opb', ''),
                features.get('opc', ''),
                features.get('opd', ''),
            ]).strip()
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'medqa':
            features = ast.literal_eval(row['features'])
            input_text = features.get('question', None)
            answer = features.get('answer', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'pubmed':
            features = ast.literal_eval(row['features'])
            input_text = features.get('article', None)
            abstract = features.get('abstract', None)
            return {'input': input_text, 'output': abstract}
        elif row['dataset'] == 'pubmedqa':
            features = ast.literal_eval(row['features'])
            question = features.get('QUESTION', None)
            answer = features.get('LONG_ANSWER', None)
            return {'input': question, 'output': answer}

    dataframe['for_sampling'] = dataframe.apply(extract_for_sampling, axis=1)
    return dataframe

diabetes3 = add_input_to_for_sampling(diabetes2)

In [25]:
final_combined_train_sample[final_combined_train_sample['task']=='generation'].iloc[2].tolist()

[2002,
 'icliniq',
 'train',
 "{'input': 'Hello doctor,I experienced difficulty in retracting the foreskin and pulling it down over the penis head (mild phimosis). With difficulty, I manage in 10 minutes. Do this fornightly to clean the penis. I wish to avoid circumcision and understand the application of Lignocaine gel will help. Should I apply it on penis gland or head or on top of the foreskin?', 'answer_icliniq': 'Hi. I can understand your concern about your symptoms. The application is a local anesthetic. Wherever you have pain when you retract foreskin (mostly skin will have pain). You can apply Lignocaine. You cannot apply Lignocaine inside of your foreskin since you have phimosis. So the external application is only possible. Considering your age, kindly rule our other causes of phimosis. Keep blood sugar under control and take antibiotics and apply the local moisturizing gel.  Take care.', 'answer_chatgpt': 'It is important to consult with a healthcare professional before atte

In [26]:
import ast

def parse_features(row):
    try:
        return ast.literal_eval(row)
    except (ValueError, SyntaxError):
        return {}
        
def generate_input_output(row):
    input_value = ""; output_value = ""

    task = row['task']
    features = parse_features(row['features'])

    if row['dataset']=='bionli' and task=='nli':
        query = features.get('query', '') 
        input_value = query.split("INPUT: ")[1].split("[HYP]")[0].strip() if "INPUT: " in query else None
        output_value = features.get('answer', None)
    elif row['dataset']=='biorel' and task=='ie_extract_relation':
        input_value = features.get('text', None)
        output_value = features.get('relation', None)
    # elif row['dataset']=='biorel' and task=='ie_identify_relation_exist':
    #     input_value = features.get('text', '')
    #     output_value = 'YES' if features.get('relation') else 'NO'
    # elif row['dataset']=='biorel' and task=='ie_classify_relation':
    #     input_value = features.get('text', '')
    #     output_value = features.get('relation', 'unknown')
    # elif row['dataset']=='biorel' and task=='ie_generate_relation_pair':
    #     input_value = features.get('text', '')
    #     h_name = features.get('h', {}).get('name', 'unknown')
    #     t_name = features.get('t', {}).get('name', 'unknown')
    #     relation = features.get('relation', 'unknown')
    #     output_value = f"Entity Pair: ('{h_name}', '{t_name}'), Relationship: '{relation}'"
    elif row['dataset']=='icliniq' and task=='generation':
        input_value = features.get('input', None)
        output_value = features.get('answer_icliniq', None)
    elif row['dataset']=='medqa' and task=='qa_objective':
        question = features.get('question', None)
        options = features.get('options', None)
        answer = features.get('answer', None)
        answer_idx = features.get('answer_idx', None)

        input_value = (
            f"{question} Please select one of the following: A) {options['A']}, B) {options['B']}, C) {options['C']}, D) {options['D']}."
        )
        output_value = (
            f"{answer_idx}) {answer}"
        )
    elif row['dataset']=='medmcqa' and task=='qa_subjective': 
        question = features.get('question', None)
        options = {
            'A': features.get('opa', '').strip(),
            'B': features.get('opb', '').strip(),
            'C': features.get('opc', '').strip(),
            'D': features.get('opd', '').strip(),
        }
        answer_num = features.get('cop', None)
        answer_idx = 'A' if answer_num == 0 else 'B' if answer_num == 1 else 'C' if answer_num == 2 else 'D'
        answer = features.get('opa', '').strip() if answer_num == 0 else features.get('opb', '').strip() if answer_num == 1 else features.get('opc', '').strip() if answer_num == 2 else features.get('opd', '').strip()

        input_value = (
            f"{question} Please select one of the following: A) {options['A']}, B) {options['B']}, C) {options['C']}, D) {options['D']}."
        )
        output_value = (
            f"{answer_idx}) {answer}"
        )

    # elif row['dataset']=='pubmedqa' and task=='qa_mesh_tagging':
    #     features = ast.literal_eval(row['features'])  
    #     question = features.get('question', '').strip()
    #     context = " ".join(features.get('context', {}).get('contexts', [])).strip()
    #     meshes = features.get('context', {}).get('meshes', [])
        
    #     input_value = (
    #         f"Question: {question} "
    #         f"Context: {context}"
    #     )
    #     output_value = f"MeSH Tags: {', '.join(meshes)}"
    # elif row['dataset']=='pubmedqa' and task=='qa_complex_answer':
    #     features = ast.literal_eval(row['features'])  
    #     question = features.get('question', '').strip()
    #     context = " ".join(features.get('context', {}).get('contexts', [])).strip()
    #     short_answer = features.get('final_decision', '').strip()  
    #     long_answer = features.get('long_answer', '').strip()  

    #     input_value = (
    #         f"Question: {question} "
    #         f"Context: {context}"
    #     )
    #     
    #     output_value = (
    #         f"Short Answer: {short_answer} "
    #         f"Long Answer: {long_answer}"
    #     )
    # elif row['dataset']=='pubmedqa' and task=='qa_short_answer':
    #     features = ast.literal_eval(row['features'])  
    #     question = features.get('question', '').strip()
    #     context = " ".join(features.get('context', {}).get('contexts', [])).strip()
    #     short_answer = features.get('long_answer', '').strip()  

    #     input_value = (
    #         f"Question: {question} "
    #         f"Context: {context}"
    #     )
    #     output_value = f"Answer: {short_answer}"
    elif row['dataset']=='pubmedqa' and task=='qa_descriptive':
        question = features.get('QUESTION', '').strip()
        context = " ".join(features.get('CONTEXTS'))
        long_answer = features.get('LONG_ANSWER', '').strip() 

        input_value = (
            f"Question: {question} "
            f"Context: {context}"
        )
        output_value = f"{long_answer}"

    elif row['dataset']=='pubmed' and task=='summarization':
        article = features.get('article', '').strip()  
        abstract = features.get('abstract', '').strip() 
        
        input_value = f"{article}"
        output_value = abstract
    return input_value, output_value

final_combined_train_sample[['input', 'output']] = final_combined_train_sample.apply(
    lambda row: pd.Series(generate_input_output(row)), axis=1
)

final_combined_test_sample[['input', 'output']] = final_combined_test_sample.apply(
    lambda row: pd.Series(generate_input_output(row)), axis=1
)

In [29]:
print(final_combined_train_sample['dataset'].unique())
print('')
print(final_combined_train_sample['task'].unique())

['bionli' 'biorel' 'icliniq' 'medmcqa' 'medqa' 'pubmedqa' 'pubmed']

['nli' 'ie_extract_relation' 'generation' 'qa_subjective' 'qa_objective'
 'qa_descriptive' 'summarization']


In [30]:
import pandas as pd

final_df = pd.concat([final_combined_train_sample, final_combined_test_sample])

In [31]:
final_combined_train_sample.to_json("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/train_instruction_dataset.json", orient="columns", indent=4)
final_combined_test_sample.to_json("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/test_instruction_dataset.json", orient="columns", indent=4)